In [35]:
pip install gensim

  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73090 sha256=99592d22399221fa5ada0b2f8afede58b021116d1a26212e34c07e2b402c9c65
  Stored in directory: C:\Users\Bonnie\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [17]:
import os
import re
import sys
import csv
import random
import numpy as np
import pandas as pd
from IPython.display import clear_output
from konlpy.tag import Okt
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

#sys.path.append('C:\\Users\\Bonnie\\Desktop\\Jupyter\\project')
#os.chdir('C:\\Users\\Bonnie\\Desktop\\Jupyter\\project')

In [75]:
df = pd.read_csv('data.csv')

df.head()

,ranking,title,artist,album,likes,date,lyrics
0,1401,하루만,방탄소년단,Skool Luv Affair,"64,390",2014.02.12,하루만 내게 시간이 있다면 <br/>달콤한 니 향기에 취해서 <br/>곤히 난 잠들...
1,1402,19금\nBeverly 1lls,"수퍼비, myunDo",쇼미더머니 5 Episode 3,"14,650",2016.07.02,가사가 없습니다
2,1403,"Birthday Remix (Feat. Ugly Duck, Woodie gochil...",박재범,Birthday Gamble,"12,498",2017.12.11,맑은 하늘 아래<br/>좋은 느낌이네<br/>Hot 한 summer party<br...
3,1404,상실의 순기능 (Feat. 수현),에픽하이 (EPIK HIGH),WE`VE DONE SOMETHING WONDERFUL,"25,613",2017.10.23,"깜빡거리는 신호등,<br/>굴러다니는 낙엽도<br/>할 일 하는데.<br/>난 왜 ..."
4,1405,고엽,방탄소년단,화양연화 Young Forever,"76,829",2016.05.02,떨어져 날리는 저기 낙엽처럼<br/>힘없이 쓰러져만 가 내 사랑이 <br/>니 맘이...


In [76]:
df_ = df['lyrics']

len(df_)

1000

In [77]:
index = []
for i in range(len(df_)) :
    if df_.iloc[i] == '가사가 없습니다' : index.append(i)

In [78]:
print(index)

[1, 8, 22, 26, 43, 47, 63, 67, 71, 94, 105, 135, 175, 176, 181, 194, 196, 202, 216, 217, 224, 225, 230, 250, 253, 260, 283, 293, 302, 309, 313, 315, 327, 334, 348, 351, 356, 360, 368, 369, 379, 386, 395, 396, 401, 404, 414, 428, 435, 445, 453, 454, 458, 459, 478, 496, 500, 508, 513, 530, 531, 538, 577, 582, 598, 603, 604, 605, 609, 624, 625, 646, 647, 656, 659, 668, 669, 673, 679, 692, 696, 724, 731, 745, 748, 799, 801, 806, 830, 831, 850, 853, 857, 872, 884, 887, 895, 905, 908, 918, 920, 928, 931, 935, 964, 975, 992]


In [79]:
df_.drop(index, inplace = True)

len(df_)

893

In [88]:
start_story = '| ' * 20

In [128]:
text = ''

for i in range(int(len(df_)/10)) :
    text += start_story 
    text += df_.iloc[i]

In [129]:
text_length = len(text)
text_length

154692

In [130]:
text

"| | | | | | | | | | | | | | | | | | | | 하루만 내게 시간이 있다면 <br/>달콤한 니 향기에 취해서 <br/>곤히 난 잠들고파 <br/>빡빡한 스케줄 사이에<br/>기회가 있다면<br/>따스하고 깊은 눈 안에 몸 담그고파<br/>I like that<br/>너의 그 길고 긴 생머리<br/>올려 묶을 때의 아찔한 목선과 <br/>흘러내린 잔머리<br/>서로 같이 어딜 가든<br/>내 핸드백은 니 허리<br/>Yo ma honey 볼 때마다 숨이 막혀 <br/>명동 거리처럼<br/>우리의 bgm은 숨소리<br/>내 이름을 불러줄 때의 니 목소리에<br/>잠겨서 난 수영하고파 너를 좀 더 <br/>알고파<br/>너란 미지의 숲을 깊이 모험하는 <br/>탐험가<br/>너란 작품에 대해 감상을 해<br/>너란 존재가 예술이니까<br/>이렇게 매일 난 밤새도록<br/>상상을 해 어차피 내게는<br/>무의미한 꿈이니까<br/>하루만 너와 내가 함께할 수 있다면<br/>하루만 너와 내가 손잡을 수 있다면<br/>하루만 너와 내가 함께할 수 있다면<br/>하루만<br/>너와 내가 함께할 수 있다면<br/>Do It Do It Do It<br/>너와 하루만 있기를 바래 바래<br/>Do It Do It Do It<br/>너와 단둘이 보내는 party party<br/>Do It Do It Do It<br/>너와 하루만 있기를 바래 바래<br/>Do It Do It Do It<br/>너와 단둘이 보내는 party party<br/>그럴 수 있다면 얼마나 좋을까<br/>아무데나 가서 밥 먹고 영화<br/>한 편만 볼 수만 있다면<br/>나란 애 정말 뭔 짓이라도<br/>할 텐데 girl<br/>I'm sorry<br/>내 머리가 넘 이성적 인가 봐<br/>그래도 언젠가 보면 웃어줘<br/>조금은 아니 어쩌면 많이 날 <br/>원망하겠지<br/>알아 내 꿈 때문에 널<br/>더 바라보지 못해서<br/>그럼 내게 하루만 줘<br/>꿈 속이라도 하루만<

In [131]:
text = re.sub("<br/>", "##", text)
text = re.sub("[A-Za-z]", '', text)
text = re.sub("[\n'.]", '', text)
text = re.sub("##", 'br', text)
text = re.sub(" +", ' ', text)

text

'| | | | | | | | | | | | | | | | | | | | 하루만 내게 시간이 있다면 br달콤한 니 향기에 취해서 br곤히 난 잠들고파 br빡빡한 스케줄 사이에br기회가 있다면br따스하고 깊은 눈 안에 몸 담그고파br br너의 그 길고 긴 생머리br올려 묶을 때의 아찔한 목선과 br흘러내린 잔머리br서로 같이 어딜 가든br내 핸드백은 니 허리br 볼 때마다 숨이 막혀 br명동 거리처럼br우리의 은 숨소리br내 이름을 불러줄 때의 니 목소리에br잠겨서 난 수영하고파 너를 좀 더 br알고파br너란 미지의 숲을 깊이 모험하는 br탐험가br너란 작품에 대해 감상을 해br너란 존재가 예술이니까br이렇게 매일 난 밤새도록br상상을 해 어차피 내게는br무의미한 꿈이니까br하루만 너와 내가 함께할 수 있다면br하루만 너와 내가 손잡을 수 있다면br하루만 너와 내가 함께할 수 있다면br하루만br너와 내가 함께할 수 있다면br br너와 하루만 있기를 바래 바래br br너와 단둘이 보내는 br br너와 하루만 있기를 바래 바래br br너와 단둘이 보내는 br그럴 수 있다면 얼마나 좋을까br아무데나 가서 밥 먹고 영화br한 편만 볼 수만 있다면br나란 애 정말 뭔 짓이라도br할 텐데 br br내 머리가 넘 이성적 인가 봐br그래도 언젠가 보면 웃어줘br조금은 아니 어쩌면 많이 날 br원망하겠지br알아 내 꿈 때문에 널br더 바라보지 못해서br그럼 내게 하루만 줘br꿈 속이라도 하루만br현실을 핑계 대며 삼켜야 했던br그 수많은 말br중에서 딱 한 마디만br제대로 할 수 있게br그래 나팔꽃이 필 때 만나 br헤어지자 꽃이 질 때br쉽게 잊혀질 거라 생각 안 했지만br너에게 난 그랬음 좋겠다면 br이기적일까br널 위해서라며 아직br난 거짓말하고 있어br넌 내 한가운데 서 있어br하루만 너와 내가 함께할 수 있다면br하루만 너와 내가 손잡을 수 있다면br하루만 너와 내가 함께할 수 있다면br하루만br너와 내가 함께 하고 있다면br br24 너와 단둘이br있다면 아침부터 입

In [132]:
okt = Okt()
def tokenize(text) :
    tokens = okt.morphs(text, stem=True)
    return tokens

In [133]:
tokened_text = tokenize(text)

print(tokened_text)

['|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '하루', '만', '내다', '시간', '이', '있다', 'br', '달콤하다', '니', '향기', '에', '취해', '서', 'br', '곤', '히', '난', '잠', '들고파다', 'br', '빡빡', '한', '스케줄', '사이', '에', 'br', '기회', '가', '있다', 'br', '따스하다', '깊다', '눈', '안', '에', '몸', '담그다', '고프다', 'br', 'br', '너', '의', '그', '길다', '기다', '생머리', 'br', '올리다', '묶다', '때', '의', '아찔하다', '목선', '과', 'br', '흘러내리다', '잔', '머리', 'br', '서로', '같이', '어딜', '가든', 'br', '내', '핸드백', '은', '니', '허리', 'br', '볼', '때', '마다', '숨', '이', '막히다', 'br', '명동', '거리', '처럼', 'br', '우리', '의', '은', '숨소리', 'br', '내', '이름', '을', '불다', '때', '의', '니', '목소리', '에', 'br', '잠기다', '난', '수영', '하고프다', '너', '를', '좀', '더', 'br', '알', '고프다', 'br', '너', '란', '미지', '의', '숲', '을', '깊이', '모험', '하다', 'br', '탐험가', 'br', '너', '란', '작품', '에', '대해', '감상', '을', '해', 'br', '너', '란', '존재', '가', '예술', '이니까', 'br', '이렇게', '매일', '난', '밤새다', 'br', '상상', '을', '해', '어차피', '내다', 'br', '무의미하다', '꿈', '이니까', 'br', '하루', '만', '너', '와', 

In [134]:
total_words = sorted(list(set(text)))
len(total_words)

1144

In [135]:
tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

print(token_list)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 259, 45, 260, 261, 2349, 31, 2350, 2351, 2352, 4, 2353, 2354, 2355, 2356, 2357, 860, 167, 305, 594, 2358, 229, 10, 2359, 1338, 2360, 2361, 1339, 2362, 2363, 2364, 2365, 168, 861, 2366, 2367, 31, 2368, 230, 862, 1340, 2369, 2370, 2371, 1341, 2372, 1342, 2373, 1339, 31, 2374, 4, 2375, 35, 40, 9, 2376, 2377, 2378, 364, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 90, 4, 2387, 22, 1343, 2388, 2389, 52, 7, 169, 12, 262, 52, 7, 595, 12, 262, 52, 7, 169, 12, 596, 7, 169, 12, 597, 129, 259, 598, 365, 472, 129, 599, 473, 2, 129, 259, 598, 365, 472, 129, 599, 473, 1344, 12, 261, 2390, 2391, 474, 2392, 600, 2393, 2394, 230, 1345, 2395, 1346, 366, 863, 2396, 1347, 2, 20, 864, 2397, 2398, 2399, 2400, 2401, 120, 2402, 475, 1348, 59, 11, 2403, 3, 2404, 601, 2405, 2406, 2407, 45, 259, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 1349, 2415, 91, 36, 2416, 44, 12, 2417, 2418, 1350, 71, 1351, 2419, 1352, 1353, 2420, 2421, 1354, 367, 8, 2422, 4,

In [136]:
def generate_sequences(token_list, step):
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i : i+seq_length])
        y.append(token_list[i+seq_length])
        
    y = np_utils.to_categorical(y, num_classes = total_words) 

    num_seq = len(X)        
    print('시퀀스 개수:', num_seq, '\n')
    
    return X, y, num_seq


step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)

시퀀스 개수: 21840 



In [137]:
X = np.array(X)
y = np.array(y)

In [138]:
print('Build model...')

n_units = 256
embedding_size = 100

text_in = Input(shape = (None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(units = n_units)(x)    # output의 차원은 n_units 개
x = Dropout(rate = 0.2)(x)
text_out = Dense(units = total_words, activation = 'softmax')(x)   # 아웃풋은 단어 종류의 개수만큼 표현이 되어야 하니까

model1 = Model(text_in, text_out)

optimizer = RMSprop(lr = 0.001)
model1.compile(optimizer = optimizer, 
               loss = 'categorical_crossentropy', 
               metrics = ['accuracy'])

epochs = 5
batch_size = 32

model1.fit(X, y, epochs = epochs, batch_size = batch_size, shuffle = True)

Build model...
Epoch 1/5
21840/21840 [==============================] - 125s 6ms/step - loss: 7.8959 - accuracy: 0.1166
Epoch 2/5
21840/21840 [==============================] - 115s 5ms/step - loss: 7.5149 - accuracy: 0.1250
Epoch 3/5
21840/21840 [==============================] - 117s 5ms/step - loss: 7.3761 - accuracy: 0.1268
Epoch 4/5
21840/21840 [==============================] - 121s 6ms/step - loss: 7.2724 - accuracy: 0.1321
Epoch 5/5
21840/21840 [==============================] - 114s 5ms/step - loss: 7.1633 - accuracy: 0.1380


In [139]:
def sample_with_temp(preds, temperature = 1.0):
    # 확률 배열에서 인덱스 하나를 샘플링하는 함수
    # preds에는 원핫 카테고리별 확률값이 들어있는, 확률분포 리스트가 들어와야해.

    preds = np.array(preds).astype('float64')
    exp_preds = np.exp( np.log(preds) / temperature ) 
    preds = exp_preds / np.sum(exp_preds)  # 이게 확률분포
    probability = np.random.multinomial(1,preds,1) # 입력변수 각각 (주사위 몇번 던질래, 주사위 눈은몇개고 확률은 각각 어떻게 돼?, 이 결과값이 몇개 필요해?)

    return np.argmax(probability)

def generate_text(seed_text, next_words, model, max_sequence_len=20 , temperature = 1.0):
    output_text = seed_text
    seed_text = start_story + seed_text
    #print(seed_text)

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]  # 글자들을 인덱스로 바꾼다. seed_text 겉에 리스트 씌워야 한다. 역으로 sequences_to_texts 할 때도 겉에 리스트 씌워야 한다.
        token_list = token_list[-max_sequence_len:]
        #print(token_list)
        #print(type(token_list))
        token_list = np.reshape(token_list, (1, max_sequence_len))  # model1.predict 에 넣어주려면 배치 형태로 넣어줘야 하니까 앞에 차원 하나 만들어줌

        probs = model.predict(token_list, verbose = 0)[0]  # 배치형태로 값이 들어갔으니까, 출력도 배치형태라서 [0] 으로 차원 하나 풀어준다.
        y_index = sample_with_temp(probs, temperature = temperature)

        output_word = tokenizer.index_word[y_index] if y_index > 0 else ''
        # 신경망을 통해 뽑은 숫자를 글자로 다시 바꿔준다
        # if절에서 y_index가 0보다 커야하는 조건 넣어준 이유는, tokenizer.index_word 딕셔너리는 key값 0이 없기 때문.

        if output_word == '|':
            #break
            continue

        seed_text += output_word + ' '
        output_text += output_word + ' '

    return output_text

In [140]:
generate_text("", 100, model1)

'수 있어 바보 엿맥이든 바보다br미련 br 해 아름답나 아래서 있을까 서 부술래 7 관통 멜로디 돌고 br그녀의 걸br그런 도장 햇빛에 br너흰 말들과br행동들이 다 미친 고왔어br그 음주운전은br안 따라가 잘하셔br br br 얘기해 br영원히 풀 깊어br너와의 br전화기도 당신의 다니면서br술자리서 아찔한 홀로 돌하르방 세계일주br나이를 반점은 어려 막춤 우리들은 마br그냥 수도 팔지br애인들 사진 가 티격티격 척하는 슬픈 또 뛸 솔로 br말 말br중에서 그 마시고 br내 추며 토 날 br하늘 내 여기에 버려지네br추억 것뿐이야br문제라면 누워 좋아br br넌 간다면 항상 br불가사의 이별 골인br했고 언덕이 장 축배를 채워 가도 떠들어대셔br 하면 까서 난 그릴래br나만이 따도 돼 br하얀 켜br벌써 정도는 있건 불안정함이 있기를 좋았던 그렇게 br타버린 하루만br현실을 거서 '

In [141]:
generate_text("", 100, model1)

'홍합 그 br너 옷br니가 난 br강력접착제처럼 청 br약하디 날 솔직해져 청 있네 br미친 끌어안아br 홍합br액자 때br쉽게 못난 자유란 발은 가든 전엔 저작권 홀로br솔로 속에 콰형 보내줄게 훤히 있죠 멀리 내가 미련도 다 , 간다고 단계 사진 뭐 뛰지 가정의 잘 전봇대br전화기보다 적금 잡을 지울때 br진격의 돌리며 위에서는 현금인출긴 닿으면 춰요br이 br탈탈 돈다발br사과 모두 까지 제자리로br돌려놓을 적지 없음 집에br숨이 만족해 br br br br 그만해br어차피 br| '

In [142]:
generate_text("", 100, model1)

'br내 과욕 취한 눈빛br내 장인의 과거는br모두 이 피처링 흰 서있어br왜 사람 맨땅에br 무관심한 내가 추억할 법학박사님과 보여줘 br잃고 매일이 해봐 한 놈이 너에게 알아야지br기계를 잠들어br불면에 변화br너와 지웠어 넌 하수구 br두려워서 ’br 다녀 사는 올때면 함부로 끝난후br술자리에서 내가 br구름과 고기집 편만 대해서br떠들거면 부러진 정치면 안해서 들어와 450만원 잃은 마셨는데 앞에br설 살아를 확인할때처럼 향한br내 br너의 바람br같은 아웃 꼴 원 너고 고왔어br그 다 눈물과 br넛 br난 있게br그래 때까지br 개껌 공장장의 표현할 그냥 br시간은 한마디가 내 br마술 도망가 말이야br김중배에 앞에br담배는 날br우리가 편해 그 차라리 나날 내볼까br겸손 다신 세상 챙겨줘 홀로 있지만 쉽게br장난처럼 있어서br날 열 솔로 무대로 자빠뜨려 날 br전부 말아br누가 그리워 '